#### Wrangle Spark Exercises

These exercises should go in a notebook or script named `wrangle`. Add, commit, and push your changes.

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

2. Let's see how writing to the local disk works in spark:

 - Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`.
 - Inspect your folder structure. What do you notice?


3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [ ]:
# 1. read exercise dataframes into spark dataframes
case_df = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
dept_df = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
source_df = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

In [ ]:
# write to local disk in scv and json

case_df.write.json("case.df_json", mode = "overwrite")
case_df.write.csv("case.df_csv", mode = "overwrite")

dept_df.write.json("dept.df_json", mode = "overwrite")
dept_df.write.csv("dept.df_csv", mode = "overwrite")

source_df.write.json("source.df_json", mode = "overwrite")
source_df.write.csv("source.df_csv", mode = "overwrite")

# spark proceeds to create folders for each of these dataframes for their respective types. 
# furthermore, files are split into almost equally sized files instead of one consolidated
# file.

In [ ]:
# inspect data in dataframes
# case

case_df.dtypes

In [ ]:
# inspect data in dataframes
# dept

dept_df.dtypes

In [ ]:
# inspect data in dataframes
# source

source_df.dtypes

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [ ]:
# How old is the latest (in terms of days past SLA) currently open issue?
case_df.show(1, vertical = True)

In [ ]:
case_df.select(max('case_opened_date')).show()

In [ ]:
(case_df
 .select('SLA_days', 
         'case_opened_date')
 .where(expr('! case_closed'))
 .where('case_opened_date == "2018-08-08 10:38:00"')
 .show())

In [ ]:
(case_df
 .select('SLA_days', 'case_opened_date')
 .where(expr('! case_closed'))
 .agg(expr('max(case_opened_date)')).show())

In [ ]:
# How long has the oldest (in terms of days since opened) currently opened issue been open?


2. How many Stray Animal cases are there?

In [ ]:
(case_df
 .select(count(case_df.case_id)
 .alias('stray_animal_case_count'))
 .show())

In [ ]:
(case_df
 .withColumn('stray_cases', expr('service_request_type == "Stray Animal"'))
 .select(count('stray_cases'))
 .show())

3. How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

4. Convert the `council_district` column to a string column.

In [ ]:
case_df = case_df.withColumn('council_district', col('council_district').cast('string'))
case_df.dtypes

5. Extract the year from the `case_closed_date` column.

6. Convert `num_days_late` from days to hours in new columns num_hours_late.

7. Join the case data with the source and department data.

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?

12. How do number of days late depend on department and request type?